In [17]:
import pandas as pd
import numpy as np

data = [[1, 0], [2, 1], [3, 0], [4, 2], [5, 1], [6, 0], [7, 1], [8, 3], [9, 2], [10, 1], [11, 2], [12, 1]]
heights = pd.DataFrame(data, columns=['id', 'height']).astype({'id':'Int64', 'height':'Int64'})
display(heights)

,id,height
0,1,0
1,2,1
2,3,0
3,4,2
4,5,1
5,6,0
6,7,1
7,8,3
8,9,2
9,10,1


In [19]:
heights= heights.sort_values(by='id')
heights['left_max']=heights['height'].cummax()
heights['right_max'] = heights['height'][::-1].cummax()[::-1].values
heights['min_of_right_left']= heights[['left_max','right_max']].min(axis=1)
heights['trapped']= abs(heights['height']-heights['min_of_right_left'])

display(heights)

df = pd.DataFrame({'total_trapped_water':[heights['trapped'].sum()]})
display(df)

,id,height,left_max,right_max,min_of_right_left,trapped
0,1,0,0,3,0,0
1,2,1,1,3,1,0
2,3,0,1,3,1,1
3,4,2,2,3,2,0
4,5,1,2,3,2,1
5,6,0,2,3,2,2
6,7,1,2,3,2,1
7,8,3,3,3,3,0
8,9,2,3,2,2,0
9,10,1,3,2,2,1


,total_trapped_water
0,6
